# Prueba de funciones


📥Importaciones 

In [3]:
#
# Importación del módulo warnings para controlar advertencias
import warnings
from typing import Tuple

# Importación de pandas y se asigna el alias 'pd' para su uso
import pandas as pd
import time
# Importación de funciones desde data_utils
from data_utils import ej_review_sentimiento

# Configuración de la omisión de advertencias
warnings.filterwarnings("ignore")
import pandas as pd
import pyarrow as pa

# Carga de la extensión autoreload y configuración para recargar automáticamente módulos
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


📦 Extracción de los conjuntos de datos

In [2]:
df_steam_games =   pd.read_parquet('../0 Dataset/steam_games_LISTO.parquet')
#df_user_items =   pd.read_parquet('./0 Dataset/user_items_LISTO.parquet')
#df_user_reviews = pd.read_parquet('./0 Dataset/user_review_LISTO.parquet')

##### 1 🌐 `developer(desarrollador: str)`:

Devuelve la cantidad de juegos y porcentaje de contenido Free por año según empresa desarrolladora

**📂 steam_games.json**: developer, release_date, price

In [5]:
def desarrollador(desarrollador: str):
    start_time = time.time()  # Registro del tiempo de inicio
    
    
    
    # Filtrar por desarrollador
    df_dev = df_steam_games[df_steam_games['developer'] == desarrollador]
    # Obtener la cantidad de ítems por año
    items_por_anio = df_dev.groupby('release_date').size()
    # Obtener la cantidad de ítems gratuitos por año
    gratis_por_anio = df_dev[df_dev['price'] == 0].groupby('release_date').size()
    # Calcular el porcentaje de contenido gratuito por año
    porcentaje_gratis_por_anio = (gratis_por_anio / items_por_anio * 100).round(0)
    # Reemplazar valores NaN en "Contenido Gratuito" con "0%"
    porcentaje_gratis_por_anio = porcentaje_gratis_por_anio.fillna(0)
    
    
    # Crear un DataFrame con los resultados
    result_df = pd.DataFrame({
        'Año': porcentaje_gratis_por_anio.index,
        'Juegos': items_por_anio.values,
        'Gratis %': porcentaje_gratis_por_anio.values
    })
    
    # Ordenar el DataFrame por año
    result_df = result_df.sort_values(by='Año').reset_index(drop=True)
    # Formatear los valores en la columna "Gratis %"
    result_df['Gratis %'] = result_df['Gratis %'].astype(str) + '%'
    
    return result_df

datos_desarrollador = desarrollador('Valve')
print(datos_desarrollador)

     Año  Juegos Gratis %
0   1998       1     0.0%
1   1999       1     0.0%
2   2000       2     0.0%
3   2001       1     0.0%
4   2003       1     0.0%
5   2004       5     0.0%
6   2005       1   100.0%
7   2006       2     0.0%
8   2007       3    33.0%
9   2008       1     0.0%
10  2009       1     0.0%
11  2010       2    50.0%
12  2011       1     0.0%
13  2012       2    50.0%
14  2014       7    14.0%
15  2016       1   100.0%
16  2017       2     0.0%


* Esta función desarrollador se encarga de analizar la información de los juegos en la base de datos df_steam_games, específicamente aquellos desarrollados por el desarrollador proporcionado como argumento.

* En resumen, la función realiza lo siguiente:

* Filtra los juegos por el desarrollador especificado.
* Cuenta la cantidad total de juegos por año.
* Cuenta la cantidad de juegos gratuitos por año.
* Calcula el porcentaje de juegos gratuitos sobre el total de juegos por cada año.
* Reemplaza los valores NaN (no disponibles) en el porcentaje de juegos gratuitos por 0%.
* Crea un DataFrame con los resultados, incluyendo el año, la cantidad total de juegos y el porcentaje de juegos gratuitos.
* Ordena el DataFrame por año.
* Formatea el porcentaje de juegos gratuitos para que se muestre con el símbolo de porcentaje (%).
* Devuelve el DataFrame resultante.

##### 🌐 userdata(User_id: str) : 
Dinero gastado por el usuario, porcentaje de recomendación y cantidad de items.

1. 📂 **user_reviews.json**: user_id, recommend, helpful.total
2. 📂 **users_items.json**: user_id, items_count

In [4]:

"""
_______________________________
1 DEF userdata(User_id: str):  

devolver cantidad de dinero gastado por el usuario
el porcentaje de recomendación en base a reviews.recommend 
y cantidad de items. 
_______________________________
"""
import pandas as pd
from typing import Dict

def userdata(User_id: str) -> Dict[str, float]:
    """
    Devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y la cantidad de items.
    
    Parameters:
    User_id (str): Identificador único del usuario.
    
    Returns:
    Dict[str, float]: Diccionario con los datos del usuario (dinero gastado, porcentaje de recomendación, cantidad de items).
    """
    # Leer los archivos Parquet
    user_reviews_df = pd.read_parquet('user_reviews.parquet')
    users_items_df = pd.read_parquet('users_items.parquet')
    
    # Obtener los datos del usuario
    user_data = users_items_df[users_items_df['user_id'] == User_id].iloc[0]
    
    # Obtener las revisiones del usuario
    user_reviews = user_reviews_df[user_reviews_df['user_id'] == User_id]
    
    # Calcular el total de dinero gastado y el porcentaje de recomendación
    total_spent = user_reviews['helpful.total'].sum()
    recommend_count = len(user_reviews[user_reviews['recommend']])
    total_reviews = len(user_reviews)
    recommend_percentage = (recommend_count / total_reviews) * 100 if total_reviews > 0 else 0.0
    
    return {
        "Dinero gastado": total_spent,
        "% de recomendación": recommend_percentage,
        "cantidad de items": user_data['items_count']
    }
